Man sollte immer prüfen, wie gut bei kategorischen Modellen alle Kategorien voneinander unterschieden werden und ob das neuronale Netz evtl. Schwierigkeiten bei der Unterscheidung gewisser Kategorien voneinander haben.

In [15]:
import gzip
import numpy as np
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense

def open_images(filename):
    with gzip.open(filename, "rb") as file:
        data = file.read()
        return np.frombuffer(data, dtype=np.uint8, offset=16)\
            .reshape(-1, 28, 28)\
            .astype(np.float32)


def open_labels(filename):
    with gzip.open(filename, "rb") as file:
        data = file.read()
        return np.frombuffer(data, dtype=np.uint8, offset=8)

X_train = open_images("data/fashion/train-images-idx3-ubyte.gz")
y_train = open_labels("data/fashion/train-labels-idx1-ubyte.gz")

X_train_mapped = X_train.reshape(60000, 784)
y_train_categorical = to_categorical(y_train)

model = Sequential()
model.add(Dense(100, activation="sigmoid", input_shape=(784,)))
model.add(Dense(10, activation="sigmoid"))
model.compile(optimizer="sgd", loss="categorical_crossentropy", metrics=["accuracy"])
model.fit(
    X_train_mapped,
    y_train_categorical,
    epochs=10,
    batch_size=1000
)
X_test = open_images("data/fashion/t10k-images-idx3-ubyte.gz").reshape(-1, 784)
y_test = to_categorical(open_labels("data/fashion/t10k-labels-idx1-ubyte.gz"))

Epoch 1/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3747 - loss: 1.9088
Epoch 2/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6623 - loss: 1.2285
Epoch 3/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7024 - loss: 1.0535
Epoch 4/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7272 - loss: 0.9482
Epoch 5/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7448 - loss: 0.8744
Epoch 6/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7595 - loss: 0.8141
Epoch 7/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7710 - loss: 0.7760
Epoch 8/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7783 - loss: 0.7445
Epoch 9/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7820 - loss: 0.7188
Epoch 10/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7866 - loss: 0.6997


#### Confusion Matrix

##### Daten in korrektes Format, eine "Panda Series" mappen d.h. 
- argmax Funktion auf jede Zeile anwenden -> axis: 1 gibt an, dass auf jede Zeile die argmax Funktion angewendet wird (0 wären die Spalten)
- Panda Series erstellen -> erstellt eine Liste mit einer Spalte mit allen Werten und einem Name

In [11]:
import pandas as pd

pred = model.predict(X_test)
# axis: gibt hier mit 1 an, dass auf jede Zeile die argmax Funktion angewendet wird (0 wären die Spalten)
# np.argmax(pred, axis=1)

y_actual = pd.Series(np.argmax(y_test, axis=1), name = 'actual')
y_pred = pd.Series(np.argmax(pred, axis=1), name = "pred")


313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 514us/step


##### Confusion Matrix erstellen

In [12]:
pd.crosstab(y_actual, y_pred)

pred,0,1,2,3,4,5,6,7,8,9
actual,,,,,,,,,,
0,789,5,14,89,9,1,63,0,30,0
1,6,909,11,52,11,0,9,0,2,0
2,21,1,643,6,216,0,93,0,19,1
3,48,10,10,840,37,1,46,0,8,0
4,1,3,184,51,669,0,84,0,8,0
5,1,0,0,4,0,759,0,121,14,101
6,224,3,168,58,163,0,346,0,38,0
7,0,0,0,0,0,32,0,875,1,92
8,2,2,20,14,4,5,6,11,934,2


Jede Zahl kommt 1000 mal vor. Handelt es sich um eine 6, wurde diese nur 300 mal richtig und 215 mal wurde fälschlicherweise eine 0 erkannt.
Und auch in die andere Richtung wurde 58 mal eine 6 erkannt, obwohl eine 0 zu sehen war.

<img src="assets/confusion_matrix_2.png">